# Neural Networks for Predicting Compressive Concrete Strength - AMM '24



In this notebook, we will compare the performance of a basic neural network (with iterative improvements) in the context of predicting the compressive strength of concrete based on material composition using Keras, pandas, NumPy, and scikit-learn.

We are going to use a feed forward neural network, make improvements to our model sequentially, and evaluate the impact of changes to our model on performance.

We will use the following model approach:

1. Feed Forward Neural Network Regression Model

We will evaluate our models using:

1.  Mean Squared Error - [MSE]
2.  Standard Deviation of MSE
3.  Mean of MSE

# About The Dataset


The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

We will be employing a simple neural network regression model to predict the target variable y, which in our case will be Strength.

### First we import our necessary prerequisites:

In [1]:
import tensorflow
import pandas as pd
import numpy as np
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# We surpress some expected warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#### Next we import our data, I am doing so locally. A copy of the data has been included in the folder/repository:
#### *NOTE: For the first model we are NOT using normalized data*

In [32]:
# Local copy of data included in folder/repository
concrete_data = pd.read_csv("C:/Users/amjmo/Downloads/concrete_data.csv")
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Next we will investigate some basic features of our data to ensure it is clean and ready to be worked with:

In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### It seems our data is clean and ready to work with, we will now split our data into predictors and our independent variable target:

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] 
target = concrete_data['Strength'] 
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

# Baseline Model


In [31]:
n_cols = predictors.shape[1] # Count of predictors we pass to our input_shape of our model below

##### We now create our baseline model with one hidden layer of 10 neurons, activation function ReLU, optimizer adam, and loss MSE:

In [10]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))  # One hidden layer with 10 nodes and input layer
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mean_squared_error')  # Compile with mean squared error loss
    return model

##### We will now train and test our model, with a 70/30 split in training vs test data, with 50 training epochs, and report the Std. Dev. and mean of MSE of 50 such training sessions below:

In [11]:
# List to store mean squared errors for each iteration, as we will run this training 50 times
mse_list = []  
n_repeats = 50  

for i in range(n_repeats):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    
    model = regression_model()
    
    # Training the model on the training data with 50 epochs
    model.fit(x_train, y_train, epochs=50, verbose=2)
    
    y_pred = model.predict(x_test)
    
    # Evaluating model metrics and saving to mse_list
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)
    
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

Epoch 1/50
23/23 - 2s - 75ms/step - loss: 33365.6484
Epoch 2/50
23/23 - 0s - 5ms/step - loss: 4999.5000
Epoch 3/50
23/23 - 0s - 5ms/step - loss: 1224.6888
Epoch 4/50
23/23 - 0s - 5ms/step - loss: 1121.8048
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 1033.2288
Epoch 6/50
23/23 - 0s - 7ms/step - loss: 974.5105
Epoch 7/50
23/23 - 0s - 5ms/step - loss: 918.0104
Epoch 8/50
23/23 - 0s - 8ms/step - loss: 864.1990
Epoch 9/50
23/23 - 0s - 7ms/step - loss: 813.6176
Epoch 10/50
23/23 - 0s - 7ms/step - loss: 764.6202
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 722.1130
Epoch 12/50
23/23 - 0s - 5ms/step - loss: 678.8488
Epoch 13/50
23/23 - 0s - 8ms/step - loss: 641.1437
Epoch 14/50
23/23 - 0s - 7ms/step - loss: 606.9656
Epoch 15/50
23/23 - 0s - 8ms/step - loss: 575.6133
Epoch 16/50
23/23 - 0s - 5ms/step - loss: 549.0140
Epoch 17/50
23/23 - 0s - 8ms/step - loss: 522.3125
Epoch 18/50
23/23 - 0s - 8ms/step - loss: 498.9703
Epoch 19/50
23/23 - 0s - 5ms/step - loss: 478.2030
Epoch 20/50
23/23 - 0s - 8ms/step

In [12]:
print(f"Mean of MSE over {n_repeats} runs: {mean_mse}")
print(f"Standard Deviation of MSE over {n_repeats} runs: {std_mse}")

Mean of MSE over 50 runs: 394.10135376699486
Standard Deviation of MSE over 50 runs: 429.8039154596729


### Analysis of Baseline Model

We seem to have a highly variable model, with a very high standard deviation in relation to the already substantial mean of MSE over 50 runs. This model clearly warrants improvements to improve consistency. This is in our current circumstance of NOT using normalized data, but we will see if using normalized data can improve our metrics.

# Normalized Data Implementation

##### This time we ARE using normalized data in our neural network, and we will see if this improves either of our evaluation metrics.


In [14]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head() # We have normalized our predictors

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [15]:
n_cols2 = predictors_norm.shape[1] # We will now pass the normalized predictors to the input_shape of our model below

In [16]:
def regression_model2():
    # create model
    model2 = Sequential()
    model2.add(Dense(10, activation='relu', input_shape=(n_cols2,)))  # One hidden layer with 10 nodes and input layer
    model2.add(Dense(1))  # Output layer for regression
    model2.compile(optimizer='adam', loss='mean_squared_error')  # Compile with mean squared error loss
    return model2

In [17]:
# New list to store mean squared errors for each iteration, as we will run this training 50 times
mse_list2 = []  
n_repeats = 50  

for i in range(n_repeats):
    # We now use predictors_norm
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    model2 = regression_model2()
    
    # Training the model on the training data with 50 epochs
    model2.fit(x_train, y_train, epochs=50, verbose=2)
    
    y_pred = model2.predict(x_test)
    
    # Evaluating model metrics and saving to mse_list2
    mse = mean_squared_error(y_test, y_pred)
    mse_list2.append(mse)
    
mean_mse2 = np.mean(mse_list2)
std_mse2 = np.std(mse_list2)

Epoch 1/50
23/23 - 2s - 75ms/step - loss: 1524.2992
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1508.3367
Epoch 3/50
23/23 - 0s - 5ms/step - loss: 1491.6481
Epoch 4/50
23/23 - 0s - 8ms/step - loss: 1474.8871
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 1457.1075
Epoch 6/50
23/23 - 0s - 5ms/step - loss: 1438.8639
Epoch 7/50
23/23 - 0s - 8ms/step - loss: 1419.2472
Epoch 8/50
23/23 - 0s - 7ms/step - loss: 1398.9529
Epoch 9/50
23/23 - 0s - 7ms/step - loss: 1377.3514
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 1354.5830
Epoch 11/50
23/23 - 0s - 4ms/step - loss: 1330.0404
Epoch 12/50
23/23 - 0s - 8ms/step - loss: 1304.9240
Epoch 13/50
23/23 - 0s - 7ms/step - loss: 1278.0726
Epoch 14/50
23/23 - 0s - 7ms/step - loss: 1250.2272
Epoch 15/50
23/23 - 0s - 7ms/step - loss: 1221.1864
Epoch 16/50
23/23 - 0s - 7ms/step - loss: 1191.1501
Epoch 17/50
23/23 - 0s - 5ms/step - loss: 1159.6533
Epoch 18/50
23/23 - 0s - 8ms/step - loss: 1127.4940
Epoch 19/50
23/23 - 0s - 7ms/step - loss: 1094.2474
Epoch 20/50
23/23 - 

In [26]:
print(f"Mean of MSE [Normalized] over {n_repeats} runs: {mean_mse2}")
print(f"Standard Deviation of MSE [Normalized] over {n_repeats} runs: {std_mse2}")

Mean of MSE [Normalized] over 50 runs: 355.321718046969
Standard Deviation of MSE [Normalized] over 50 runs: 77.79112150258186


### Analysis of Normalized Data Improvement

We have improved the variability of our model, with a much lower standard deviation in relation to our previous evaluation metrics when NOT using normalized data. The mean of MSE over 50 runs decreased somewhat but remains substantial, while the greatly lowered standard deviation implies a marked improvement in convergence of our model, demonstrating the importance of using normalized data for improving model performance.

# Extending Training Implementation

##### Next, we will see if we can further improve the performance of our model by increasing the number of training epochs to 100, continue using normalizaed data, and compare how this affects the mean of MSE and standard deviation of MSE of 50 runs relative to our previous model versions.

In [19]:
def regression_model3():
    # create model
    model3 = Sequential()
    model3.add(Dense(10, activation='relu', input_shape=(n_cols2,)))  # One hidden layer with 10 nodes and input layer
    model3.add(Dense(1))  # Output layer for regression
    model3.compile(optimizer='adam', loss='mean_squared_error')  # Compile with mean squared error loss
    return model3

In [20]:
# New list to store mean squared errors for each iteration, as we will run this training 50 times
mse_list3 = []  
n_repeats = 50  

for i in range(n_repeats):
    # We now use predictors_norm
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    model3 = regression_model3()
    
    # Training the model on the training data with 100 epochs
    model.fit(x_train, y_train, epochs=100, verbose=2)
    
    y_pred = model.predict(x_test)
    
    # Evaluating model metrics and saving to mse_list3
    mse = mean_squared_error(y_test, y_pred)
    mse_list3.append(mse)
    
mean_mse3 = np.mean(mse_list3)
std_mse3 = np.std(mse_list3)

Epoch 1/100
23/23 - 0s - 5ms/step - loss: 1520.5449
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 1497.3679
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 1470.0314
Epoch 4/100
23/23 - 0s - 7ms/step - loss: 1440.3745
Epoch 5/100
23/23 - 0s - 7ms/step - loss: 1407.8357
Epoch 6/100
23/23 - 0s - 7ms/step - loss: 1372.6837
Epoch 7/100
23/23 - 0s - 7ms/step - loss: 1335.4249
Epoch 8/100
23/23 - 0s - 7ms/step - loss: 1296.4208
Epoch 9/100
23/23 - 0s - 7ms/step - loss: 1256.2759
Epoch 10/100
23/23 - 0s - 7ms/step - loss: 1215.4343
Epoch 11/100
23/23 - 0s - 7ms/step - loss: 1174.7798
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 1134.7740
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 1094.6744
Epoch 14/100
23/23 - 0s - 7ms/step - loss: 1055.6771
Epoch 15/100
23/23 - 0s - 8ms/step - loss: 1017.1572
Epoch 16/100
23/23 - 0s - 8ms/step - loss: 979.5898
Epoch 17/100
23/23 - 0s - 7ms/step - loss: 942.8538
Epoch 18/100
23/23 - 0s - 7ms/step - loss: 907.0267
Epoch 19/100
23/23 - 0s - 7ms/step - loss: 872.7338
Epoch 

In [27]:
print(f"Mean of MSE [100 epochs] over {n_repeats} runs: {mean_mse3}")
print(f"Standard Deviation of MSE [100 epochs] over {n_repeats} runs: {std_mse3}")

Mean of MSE [100 epochs] over 50 runs: 48.79087364357364
Standard Deviation of MSE [100 epochs] over 50 runs: 27.027112270129237


### Analysis of Extending Training Improvements

We note further substantial improvements to our model, with even lower standard deviation and mean of MSE. The mean and standard deviation of MSE over 50 runs using 100 epcohs is much lower than both of our previous models, highlighting the importance of increasing training time for improving model performance, demonstrating increased convergence with additional training time.

# Additional Hidden Layer Implementation

##### Finally, we will investigate how much further we can improve model performance by adding additional hidden layers to our neural network. We will be adding two more hidden layers, each containing 10 neurons, and we will train this model with our normalized data across 100 epochs to be consistent with our previous model improvements. 

In [22]:
def regression_model4():
    # create model now using predictors_norm
    model4 = Sequential()
    model4.add(Dense(10, activation='relu', input_shape=(predictors_norm.shape[1],)))  # Three hidden layers with 10 nodes each 
    model4.add(Dense(10, activation='relu'))
    model4.add(Dense(10, activation='relu'))
    model4.add(Dense(1))  # Output layer for regression
    model4.compile(optimizer='adam', loss='mean_squared_error')  # Compile with mean squared error loss
    return model4

In [23]:
# New list to store mean squared errors for each iteration, as we will run this training 50 times
mse_list4 = []  
n_repeats = 50  

for i in range(n_repeats):
    # We now use predictors_norm
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    model4 = regression_model4()
    
    # Training the model on the training data with 100 epochs
    model.fit(x_train, y_train, epochs=100, verbose=2)
    
    y_pred = model.predict(x_test)
    
    # Evaluating model metrics and saving to mse_list4
    mse = mean_squared_error(y_test, y_pred)
    mse_list4.append(mse)
    
mean_mse4 = np.mean(mse_list4)
std_mse4 = np.std(mse_list4)

Epoch 1/100
23/23 - 0s - 5ms/step - loss: 33.2849
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 33.1501
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 33.0948
Epoch 4/100
23/23 - 0s - 7ms/step - loss: 33.1170
Epoch 5/100
23/23 - 0s - 7ms/step - loss: 33.0790
Epoch 6/100
23/23 - 0s - 5ms/step - loss: 33.0153
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 33.0265
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 33.0051
Epoch 9/100
23/23 - 0s - 7ms/step - loss: 32.9748
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 32.9157
Epoch 11/100
23/23 - 0s - 8ms/step - loss: 32.9141
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 32.8810
Epoch 13/100
23/23 - 0s - 8ms/step - loss: 32.9309
Epoch 14/100
23/23 - 0s - 8ms/step - loss: 32.9013
Epoch 15/100
23/23 - 0s - 4ms/step - loss: 32.8775
Epoch 16/100
23/23 - 0s - 8ms/step - loss: 32.8387
Epoch 17/100
23/23 - 0s - 5ms/step - loss: 32.8146
Epoch 18/100
23/23 - 0s - 8ms/step - loss: 32.9646
Epoch 19/100
23/23 - 0s - 5ms/step - loss: 32.7978
Epoch 20/100
23/23 - 0s - 8ms/step - los

In [28]:
print(f"Mean of MSE [Hidden Layers] over {n_repeats} runs: {mean_mse4}")
print(f"Standard Deviation of MSE [Hidden Layers] over {n_repeats} runs: {std_mse4}")

Mean of MSE [Hidden Layers] over 50 runs: 32.35857126682245
Standard Deviation of MSE [Hidden Layers] over 50 runs: 2.720377282634038


### Analysis of Additional Hidden Layers and Overall Model Improvements

We can see that using additional hidden layers in our neural network has greatly reduced the variability of our model and further reduced our mean of MSE, demonstrating how adding hidden layers to our neural network has improved the consistency of our predictions. With each improvement to our baseline model, we have shown the importance of using normalized data, extending training time, and employing more hidden layers in achieving more consistent, less variable models. I hope this notebook has shown how straightforward implementing simple neural networks with Keras can be!